In [1]:
import warnings 
warnings.filterwarnings('ignore') 
import pandas as pd
import numpy as np


## préparation du data frame

In [2]:
paris_listings = pd.read_csv('../data/paris_airbnb.csv')
paris_listings.head()

,host_response_rate,host_acceptance_rate,host_listings_count,latitude,longitude,city,zipcode,state,accommodates,room_type,bedrooms,bathrooms,beds,price,cleaning_fee,security_deposit,minimum_nights,maximum_nights,number_of_reviews
0,100%,NaN,1.0,48.83349,2.31852,Paris,75014,Île-de-France,2,Entire home/apt,0.0,1.0,0.0,$75.00,$50.00,$0.00,3,15,8
1,100%,NaN,1.0,48.85100,2.35869,Paris,75004,Ile-de-France,2,Entire home/apt,0.0,1.0,1.0,$115.00,$36.00,$0.00,1,90,173
2,100%,NaN,2.0,48.85758,2.35275,Paris,75004,Île-de-France,4,Entire home/apt,2.0,1.0,2.0,$115.00,$50.00,$200.00,10,23,243
3,100%,NaN,1.0,48.86528,2.39326,Paris,75020,Ile-de-France,3,Entire home/apt,1.0,1.0,1.0,$90.00,NaN,NaN,3,365,1
4,67%,NaN,3.0,48.85899,2.34735,Paris,75001,Île-de-France,2,Entire home/apt,1.0,1.0,1.0,$75.00,$200.00,"$1,500.00",180,365,0


In [3]:
paris_listings['price'] = paris_listings['price'].str.replace(',','')
paris_listings['price'] = paris_listings['price'].str.replace('$','')
paris_listings['price'] = paris_listings['price'].astype('float')
paris_listings.head()

,host_response_rate,host_acceptance_rate,host_listings_count,latitude,longitude,city,zipcode,state,accommodates,room_type,bedrooms,bathrooms,beds,price,cleaning_fee,security_deposit,minimum_nights,maximum_nights,number_of_reviews
0,100%,NaN,1.0,48.83349,2.31852,Paris,75014,Île-de-France,2,Entire home/apt,0.0,1.0,0.0,75.0,$50.00,$0.00,3,15,8
1,100%,NaN,1.0,48.85100,2.35869,Paris,75004,Ile-de-France,2,Entire home/apt,0.0,1.0,1.0,115.0,$36.00,$0.00,1,90,173
2,100%,NaN,2.0,48.85758,2.35275,Paris,75004,Île-de-France,4,Entire home/apt,2.0,1.0,2.0,115.0,$50.00,$200.00,10,23,243
3,100%,NaN,1.0,48.86528,2.39326,Paris,75020,Ile-de-France,3,Entire home/apt,1.0,1.0,1.0,90.0,NaN,NaN,3,365,1
4,67%,NaN,3.0,48.85899,2.34735,Paris,75001,Île-de-France,2,Entire home/apt,1.0,1.0,1.0,75.0,$200.00,"$1,500.00",180,365,0


In [4]:
np.random.seed(1)
df = paris_listings.copy()
shuffled_index = np.random.permutation(df.index)
df = df.reindex(shuffled_index)
df.head()

,host_response_rate,host_acceptance_rate,host_listings_count,latitude,longitude,city,zipcode,state,accommodates,room_type,bedrooms,bathrooms,beds,price,cleaning_fee,security_deposit,minimum_nights,maximum_nights,number_of_reviews
4740,NaN,NaN,1.0,48.88285,2.33852,Paris,75018,Île-de-France,4,Entire home/apt,1.0,1.0,2.0,65.0,$30.00,$200.00,2,1125,6
5606,NaN,NaN,1.0,48.86006,2.34768,Paris,75001,Île-de-France,3,Entire home/apt,2.0,1.5,2.0,98.0,NaN,NaN,5,1124,1
4824,100%,NaN,3.0,48.88794,2.34532,Paris,75018,Île-de-France,2,Entire home/apt,0.0,1.0,1.0,65.0,$30.00,"$1,000.00",10,1125,0
4205,60%,NaN,1.0,48.88868,2.34111,Paris,75018,Île-de-France,2,Private room,1.0,1.0,1.0,45.0,$30.00,$200.00,5,1125,83
3228,90%,NaN,1.0,48.88686,2.33670,Paris,75018,Île-de-France,3,Entire home/apt,1.0,1.0,2.0,65.0,NaN,$200.00,5,365,5


### division du data en 2 partie égaux

In [5]:
split_one = df.loc[0:4000]
split_two = df.loc[4000:]

## validation croisé holdout

In [6]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
knn= KNeighborsRegressor()

train_one = split_one
test_one = split_two

train_two = split_two
test_two =split_one
# premiére moitié
knn.fit(train_one[['accommodates']],train_one['price'])
test_one['predicted'] = knn.predict(test_one[['accommodates']])
iteration_one_rmse = np.sqrt(mean_squared_error(test_one['price'],test_one['predicted']))
# second moitié
knn.fit(train_two[['accommodates']],train_two['price'])
test_two['predicted'] = knn.predict(test_two[['accommodates']])
iteration_two_rmse = np.sqrt(mean_squared_error(test_two['price'],test_two['predicted']))

avg_rmse = np.mean([iteration_one_rmse,iteration_two_rmse])
print(iteration_one_rmse,iteration_two_rmse,avg_rmse)

69.13338881665908 90.81848585908804 79.97593733787356


## validation croisé K-Fold 

In [7]:
df['Fold'] = 0
df['Fold'].iloc[0:1600]=1
df['Fold'].iloc[1600:3200]=2
df['Fold'].iloc[3200:4800]=3
df['Fold'].iloc[4800:6400]=4
df['Fold'].iloc[6400:]=5
df['Fold'].value_counts()

Fold
1    1600
2    1600
3    1600
4    1600
5    1600
Name: count, dtype: int64

#### première itération

In [8]:
models = KNeighborsRegressor()
train_one = df[df['Fold']!=1]
test_one = df[df['Fold']==1]
models.fit(train_one[['accommodates']],train_one['price'])

label=models.predict(test_one[['accommodates']])
iteration_one_mse = mean_squared_error(test_one['price'],label)
iteration_one_rmse = np.sqrt(iteration_one_mse)
print(iteration_one_rmse)

94.15347709989261


In [9]:
fold_list = [1,2,3,4,5]

def train_and_validate(df,n_fold):
    fold_rmse =[]
    for i in n_fold:
        train_n = df[df['Fold'] != i]
        test_n = df[df['Fold'] == i]
        models.fit(train_n[['accommodates']],train_n['price'])
        labels = models.predict(test_n[['accommodates']])
        test_n['predicted_price'] = labels
        mses = mean_squared_error(test_n['price'],test_n['predicted_price'])
        rmses = np.sqrt(mses)
        fold_rmse.append(rmses)
    return (fold_rmse)

rmses = train_and_validate(df,fold_list)
print(rmses)

avg_rme = np.mean(rmses)
print(avg_rme)

[94.15347709989261, 163.78089609292044, 80.19681602158529, 108.48121934233593, 83.82995273170563]
106.08847225768798


## Implémentation du validation croisé en utilisant sklearn

In [10]:
from sklearn.model_selection import cross_val_score, KFold
kf =KFold(5,shuffle = True, random_state=1)
knn = KNeighborsRegressor()
mse = cross_val_score(knn,paris_listings[['accommodates']],paris_listings['price'],
                     scoring='neg_mean_squared_error', cv= kf)
rmse = np.sqrt(np.abs(mse))
avg_rmse = np.mean(rmse)

print(rmse)
print(avg_rmse)

[ 80.90884609 172.42582434  80.27004407  98.92942876  74.6177107 ]
101.43037079170577
